In [ ]:
import numpy as np
import cv2
import scipy.misc
from skimage import color
import torch
from IPython import embed
from scipy.ndimage.interpolation import zoom
import matplotlib.pyplot as plt
import model_rec as model

%matplotlib inline

In [2]:
l_norm = 1.
l_cent = 50.
ab_norm = 1.
mask_cent = 0.
mask_mult = 110.
use_gpu = False
H_proc, W_proc = (256,256) # resolution to process, needs to be multiple of 8

def np2tens(in_np,use_gpu=True):
	# numpy HxWxC ==> Torch tensor 1xCxHxW
	out_tens = torch.Tensor(in_np.transpose((2,0,1)))[None,:,:,:]
	if(use_gpu):
		out_tens = out_tens.cuda()
	else:
		out_tens = out_tens.cpu()
	return out_tens

def tens2np(in_tens,use_gpu=True):
	# Torch tensor 1xCxHxW ==> numpy HxWxC
	if(use_gpu):
		return in_tens.cpu().numpy().transpose((2,3,1,0))[:,:,:,0]
	else:
		return in_tens.numpy().transpose((2,3,1,0))[:,:,:,0]

def ind2sub(ind, H=256, W=256):
    return (ind/H, ind - ind/H*H)

In [3]:
# ***** LOAD COLORIZER MODEL *****
colorizer = model.SIGGRAPHGenerator()
colorizer.load_state_dict(torch.load('./models/caffemodel_unnorm.pth'))
colorizer.cuda() if(use_gpu) else colorizer.cpu()
colorizer.eval();

In [4]:
def run_model(in_ab, in_mask):
    # normalize & center input ab, input mask
    in_ab_norm = in_ab/ab_norm
    in_mask_norm = (in_mask - mask_cent)*mask_mult

    out_class, out_reg = colorizer.forward(np2tens(img_rs_l_norm,use_gpu=use_gpu), np2tens(in_ab_norm,use_gpu=use_gpu), np2tens(in_mask_norm,use_gpu=use_gpu))
    out_class = out_class.data # 1 x AB x H_proc x W_proc, probability distribution at every spatial location (h,w) of possible colors
    out_reg = out_reg.data # 1 x 2 x H_proc x W_proc

    out_ab_norm = tens2np(out_reg)
    out_ab = out_ab_norm*110 # un-normalize

    # ***** CONCATENATE WITH INPUT *****
    out_orig_ab = zoom(out_ab, (1.*H_orig/H_proc, 1.*W_orig/W_proc, 1) )

    # concatenate with L channel, convert to RGB, save
    out_orig_lab = np.concatenate((img_orig_l,out_orig_ab),axis=2)
    out_orig_rgb = np.uint8(np.clip(color.lab2rgb(out_orig_lab),0,1)*255)

    return (out_orig_rgb, out_class, out_reg)

In [5]:
# ***** LOAD IMAGE, PREPARE DATA *****
# for (ii,img_path) in enumerate(['image013.jpg','image015.jpg','image017.jpg','image019.jpg','image021.jpg','image023.jpg','IMG_0217.jpg','IMG_4818-1.jpg']):
for (ii,img_path) in enumerate(['image013.jpg','image015.jpg','image017.jpg','image019.jpg','image021.jpg','image023.jpg','IMG_0217.jpg','IMG_4818-1.jpg']):
    img_orig = cv2.imread('./imgs/%s'%img_path)[:,:,::-1]
    (H_orig,W_orig) = img_orig.shape[:2]
    print('[%ix%i] Original resolution'%(H_orig,W_orig))
    print('[%ix%i] Processed resolution'%(H_proc,W_proc))

    # take L channel at fullres
    img_orig_lab = color.rgb2lab(img_orig)
    img_orig_l = img_orig_lab[:,:,[0]]

    # resize to processing size, take L channel for input
    img_rs = cv2.resize(img_orig, (W_proc, H_proc), interpolation=cv2.INTER_CUBIC)
    img_rs_lab = color.rgb2lab(img_rs)
    img_rs_l_norm = (img_rs_lab[:,:,[0]]-l_cent)/l_norm # normalized

    in_ab = np.zeros((H_proc, W_proc, 2)) # initialize blank ab input, mask
    in_mask = np.zeros((H_proc, W_proc, 1))

    (out_orig_rgb, out_class, out_reg) = run_model(in_ab, in_mask)

    plt.imsave('out_%02d.jpg'%ii,out_orig_rgb)

[477x636] Original resolution
[256x256] Processed resolution


/Users/rizhang/anaconda2/lib/python2.7/site-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))
/Users/rizhang/anaconda2/lib/python2.7/site-packages/torch/nn/functional.py:2423: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


[388x648] Original resolution
[256x256] Processed resolution
[884x586] Original resolution
[256x256] Processed resolution
[841x561] Original resolution
[256x256] Processed resolution
[428x629] Original resolution
[256x256] Processed resolution
[788x523] Original resolution
[256x256] Processed resolution
[479x720] Original resolution
[256x256] Processed resolution
[1024x791] Original resolution
[256x256] Processed resolution
